In [4]:
import mfem.ser as mfem
import mfem.ser as mfem
from mfem.ser import Mesh
import glvis as vis

In [5]:
import mfem.ser as mfem

# Create a square mesh
mesh = mfem.Mesh(10, 10, "QUADRITERAL", True, 1,1)

# Define the finite element function space
fec = mfem.H1_FECollection(2, mesh.Dimension())   # H1 order=1
fespace = mfem.FiniteElementSpace(mesh, fec)

# Define the essential dofs
ess_tdof_list = mfem.intArray()
ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

# Define constants for alpha (diffusion coefficient) and f (RHS)
alpha = mfem.ConstantCoefficient(1.0)
rhs = mfem.ConstantCoefficient(1.0)

"""
Note
-----
In order to represent a variable diffusion coefficient, you
must use a numba-JIT compiled function. For example:

>>> @mfem.jit.scalar
>>> def alpha(x):
>>>     return x+1.0
"""

# Define the bilinear and linear operators
a = mfem.BilinearForm(fespace)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(alpha))
a.Assemble()
b = mfem.LinearForm(fespace)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(rhs))
b.Assemble()

# Initialize a gridfunction to store the solution vector
x = mfem.GridFunction(fespace)
x.Assign(0.0)

# Form the linear system of equations (AX=B)
A = mfem.OperatorPtr()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)
print("Size of linear system: " + str(A.Height()))

# Solve the linear system using PCG and store the solution in x
AA = mfem.OperatorHandle2SparseMatrix(A)
M = mfem.GSSmoother(AA)
mfem.PCG(AA, M, B, X, 1, 200, 1e-12, 0.0)
a.RecoverFEMSolution(X, b, x)

# Extract vertices and solution as numpy arrays
verts = mesh.GetVertexArray()
sol = x.GetDataArray()

# Plot the solution using matplotlib
import matplotlib.pyplot as plt
import matplotlib.tri as tri

triang = tri.Triangulation(verts[:,0], verts[:,1])

fig, ax = plt.subplots()
ax.set_aspect('equal')
tpc = ax.tripcolor(triang, sol, shading='gouraud')
fig.colorbar(tpc)
plt.show()

TypeError: Wrong number or type of arguments for overloaded function 'new_Mesh'.
  Possible C/C++ prototypes are:
    mfem::Mesh::Mesh()
    mfem::Mesh::Mesh(mfem::Mesh const &,bool)
    mfem::Mesh::Mesh(mfem::Mesh &&)
    mfem::Mesh::Mesh(mfem::real_t *,int,int *,mfem::Geometry::Type,int *,int,int *,mfem::Geometry::Type,int *,int,int,int)
    mfem::Mesh::Mesh(int,int,int,int,int)
    mfem::Mesh::Mesh(std::string const &,int,int,bool)
    mfem::Mesh::Mesh(std::istream &,int,int,bool)
    mfem::Mesh::Mesh(mfem::Mesh *[],int)
    mfem::Mesh::Mesh(int,int,int,mfem::Element::Type,bool,mfem::real_t,mfem::real_t,mfem::real_t,bool)
    mfem::Mesh::Mesh(int,int,mfem::Element::Type,bool,mfem::real_t,mfem::real_t,bool)
    mfem::Mesh::Mesh(int,mfem::real_t)
    mfem::Mesh::Mesh(mfem::Mesh *,int,int)
    mfem::Mesh::Mesh(int,int,int,char const *,bool,double,double,double,bool)
    mfem::Mesh::Mesh(int,int,char const *,bool,double,double,bool)
